# Ingest Beeswax & Ipon Reports (API)

## Imports (& Load ENV variables)

In [ ]:
import requests
import time
from os import environ as ENV
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from io import StringIO

load_dotenv()

## Initialise Spark Session

In [ ]:
spark = SparkSession.builder \
    .appName("Reporting_Test") \
    .getOrCreate()

## Beeswax API Authenticate

In [ ]:
auth_url = ENV['BEESWAX_BASE_URL']+'/authenticate'
auth_payload = {'email': ENV['BEESWAX_EMAIL'], 'password': ENV['BEESWAX_PASSWORD']}
auth_headers = {'Content-Type': 'application/json'}

# authenticate request

response = requests.post(auth_url, json=auth_payload, headers=auth_headers)
response.raise_for_status()

print(response.json())

# store auth cookie in session

session = requests.Session()

auth_response = session.post(auth_url, json=auth_payload, headers=auth_headers)
auth_response.raise_for_status()


## Beeswax API Call (Get Report)

In [22]:
def bw_api_get_report(report_type, field_list, filter_dict):
    """retrieve beeswax report"""
    report_url = ENV['BEESWAX_BASE_URL']+'/v2/reporting/run-query'
    report_payload={
        'view': report_type,
        'fields': field_list,
        'filters': filter_dict,
        'result_format': 'csv'
    }

    report_response = session.post(report_url, json=report_payload)
    report_response.raise_for_status()
    data = report_response.json()

    task_id = data['task_id']
    async_url = ENV['BEESWAX_BASE_URL']+'/v2/reporting/async-results/'+task_id

    start_time = time.time()
    max_wait_seconds = 60

    while True:
        result_response = session.get(async_url)
        if result_response.status_code == 200:
            rdd = spark.sparkContext.parallelize(result_response.text.splitlines())
            report_df = spark.read.option('header', True).option('inferSchema', True).csv(rdd)
            break
        elif result_response.status_code == 202:
            elapsed_time = time.time() - start_time
            if elapsed_time > max_wait_seconds:
                raise TimeoutError (f'Report not ready after {max_wait_seconds} seconds')
            time.sleep(5)
        else:
            result_response.raise_for_status()

    return report_df




## Get Performance and Bid Performance Report Data

In [25]:
performance_report_df = bw_api_get_report(
    'performance_agg',
    [
        "bid_day", 
        "advertiser_id", 
        "advertiser_name", 
        "campaign_id", 
        "campaign_name", 
        "line_item_id", 
        "line_item_name", 
        "creative_id", 
        "creative_name", 
        "creative_size",
        "impression", 
        "clicks", 
        "media_spend", 
        "postback_conversions", 
        "viewthrough_conversions", 
        "clickthrough_conversions", 
        "video_plays", 
        "video_q1s", 
        "video_midpoints", 
        "video_q3s", 
        "video_completes", 
        "cpm", 
        "cpc", 
        "spend_per_conversion", 
        "ctr"
    ],
    {
        'bid_day': '2025-09-26'
    }
)

bid_performance_report_df = bw_api_get_report(
    'bid_performance_agg',
    [
        "bid_day", 
        "advertiser_id", 
        "campaign_id", 
        "line_item_id",  
        "creative_id", 
        "bid",
        "win_rate"
    ],
    {
        'bid_day': '2025-09-26'
    }
)



## Beeswax Join Bids & Wins to Report

## Beeswax Write to Staging Table

## Ipon API Call

## Ipon Write to Staging Table